<a href="https://colab.research.google.com/github/yantiputri/SEMESTER-3/blob/main/latihan1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut, cross_val_score, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Membuat Dataset (sesuai tabel pada soal)

In [ ]:
data = {
    "temperature": [10, 25, 15, 20, 18, 20, 22, 24],
    "wind":        [0,  0,  5,  3,  7, 10,  5,  6],
    "label":      ["Dingin","Panas","Dingin","Panas","Dingin","Dingin","Panas","Panas"]
}

df = pd.DataFrame(data)
print(df)


   temperature  wind   label
0           10     0  Dingin
1           25     0   Panas
2           15     5  Dingin
3           20     3   Panas
4           18     7  Dingin
5           20    10  Dingin
6           22     5   Panas
7           24     6   Panas


Encoding Label (Dingin 0, Panas 1)

In [ ]:
le = LabelEncoder()
y = le.fit_transform(df["label"])
X = df[["temperature", "wind"]].values

print("Label Mapping :", dict(zip(le.classes_, le.transform(le.classes_))))


Label Mapping : {'Dingin': np.int64(0), 'Panas': np.int64(1)}


Menentukan Nilai K Terbaik dengan Leave-One-Out CV

In [ ]:
# LOOCV untuk memilih k
loo = LeaveOneOut()
k_values = list(range(1, len(X)))  # k = 1 s/d 7
cv_scores = []

for k in k_values:
    model = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(model, X, y, cv=loo, scoring='accuracy')
    cv_scores.append(scores.mean())

# Menentukan k terbaik
best_k = k_values[int(np.argmax(cv_scores))]
print("K terbaik =", best_k)


K terbaik = 1


Melatih Model Dengan K Terbaik

In [ ]:
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X, y)


KNeighborsClassifier(n_neighbors=1)

Prediksi Persepsi Marry untuk (16°C, 3 km/jam)

In [ ]:
test_point = np.array([[16, 3]])
pred = knn.predict(test_point)
pred_label = le.inverse_transform(pred)[0]

print("Prediksi Marry pada (16°C, 3 km/jam) :", pred_label)


Prediksi Marry pada (16°C, 3 km/jam) : Dingin


Menampilkan Probabilitas Prediksi

In [ ]:
proba = knn.predict_proba(test_point)[0]
print("Probabilitas (Dingin, Panas) :", proba)


Probabilitas (Dingin, Panas) : [1. 0.]


Evaluasi Model (Confusion Matrix & Classification Report)

In [ ]:
y_pred = cross_val_predict(KNeighborsClassifier(n_neighbors=best_k), X, y, cv=loo)

print("Akurasi LOOCV :", accuracy_score(y, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y, y_pred))
print("\nClassification Report:\n", classification_report(y, y_pred, target_names=le.classes_))


Akurasi LOOCV : 1.0

Confusion Matrix:
 [[4 0]
 [0 4]]

Classification Report:
               precision    recall  f1-score   support

      Dingin       1.00      1.00      1.00         4
       Panas       1.00      1.00      1.00         4

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8

